## IMPUTE INSANITY!:
### before we move onto neural networks we need to impute the NANs
#### How do we even know we've correctly imputed?
#### **Try out every impute methods and per feature set train the GBM and see which one got most importance**

### TODO
- #### [KNN , DeepLearner, other novel ways to impute](https://towardsdatascience.com/6-different-ways-to-compensate-for-missing-values-data-imputation-with-examples-6022d9ca0779)
  - KNN thoughts: we can perhaps use subset of features with no nans, but then the records will become similar, and decision trees already predict the same result for similar looking rows? then what's the point of imputing value same as similar looking rows?
- ### DON't Impute rather provide the Flag wether the feature has value or not, kinda like mask; we did this in Optiver (not the transformer sequence mask , but we have mask about weather the second/grouped seconds is missing or not)
  - #### [Results on passing information about missing values to neural networks](https://machinelearningmastery.com/binary-flags-for-missing-values-for-machine-learning/)
- ### AMEND ABOVE: Impute as well as pass the missing Flag too, so neural network learns better
- ### Categorical Imputation: train on data with missing categorical values as target variable! **We can even use TEST set to do this!!!! yay!**
  - #### [Training to find missing categorical values](https://www.analyticsvidhya.com/blog/2021/04/how-to-handle-missing-values-of-categorical-variables/)

### IMPORTANT FLOAT16 aggregations return NONE!!! mean, sum most of it doesn't work :-/

## TODO:
- ### MULTILEVEL IMPUTATION DEFAULT
- ### LEVEL 1: out of all the variant+original; select the one with best amex scores DEFAULT_EMP1
    - #### This is partial nans
- ### Level 2: now that we have partial none df with best variants(including original with nans) ; TRAIN ML MODEL to have PREDICTED/BS variant
    - #### In the order of lowest missing values to highest missing values
    - #### we Target the feature with lowest amount on NAN as we have maximum training rows; then slowly cascade to more missing row features
    - #### once we have computed PREDICTED/BS variants for all of the features with nans
- ### Level 3: from DEFAULT_EMP1 vs PREDICTED variant select the one with maximum amex score (this is kinda like back to stage 1 but with improved train_df_wt)
    - #### repetetively select and choose the best varint out of the two
    - #### DEFAULT_EMP2 is generated!
    - #### Now there is a scope of repeating Level 2 again with using base_df of EMP2 and then going to EMP3,

In [1]:
WHERE_THIS = "local" # local|kaggle

BS_DATASET = "amex-default-prediction-binarysentient"
if WHERE_THIS == "kaggle":
    INPUT_PATH = "/kaggle/input/amex-default-prediction/"
    OUTPUT_PATH = "/kaggle/working/"
    TEMP_PATH = "/kaggle/temp/"
elif WHERE_THIS == "local":
    INPUT_PATH = "input/amex-default-prediction"
    OUTPUT_PATH = "working"
    TEMP_PATH = "temp"
import os
BS_PATH = os.path.join(TEMP_PATH,BS_DATASET)

In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import gc
import os
from datetime import datetime
import torch


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.display import display, HTML
from lightgbm import LGBMClassifier, log_evaluation, LGBMRegressor, r2_score
from sklearn.model_selection import StratifiedKFold, KFold
import plotly.express as pltex

import impute_insanity
from impute_insanity import load_prepare_amex_dataset
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


for dirname, _, filenames in os.walk(INPUT_PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ImportError: cannot import name 'r2_score' from 'lightgbm' (C:\Users\nisha\miniconda3\envs\kaggle\lib\site-packages\lightgbm\__init__.py)

In [5]:
object_features = ['customer_ID']
discard_features = ['D_66']
datetime_features = ['S_2']
cat_features = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_68']
target_features = ['target']
numeric_features = ['R_2', 'S_25', 'D_118', 'B_5', 'D_60', 'B_21', 'D_115', 'S_15', 'D_84', 'D_122', 'B_9', 'B_33', 'D_53', 'R_24', 'D_94', 'D_56', 'D_139', 'R_28', 'B_3', 'S_20', 'B_31', 'D_133', 'B_2', 'S_12', 'D_71', 'D_96', 'S_7', 'D_72', 'B_36', 'B_41', 'S_5', 'D_41', 'R_22', 'R_8', 'D_140', 'D_47', 'D_89', 'P_2', 'R_19', 'D_59', 'B_23', 'S_3', 'D_145', 'D_103', 'B_19', 'R_20', 'D_73', 'D_136', 'D_141', 'D_142', 'B_22', 'D_46', 'B_29', 'B_25', 'D_128', 'B_18', 'D_86', 'D_109', 'B_8', 'B_17', 'R_17', 'B_12', 'D_54', 'D_74', 'S_16', 'B_6', 'D_49', 'D_80', 'S_8', 'B_7', 'D_144', 'B_27', 'B_26', 'R_25', 'R_23', 'D_82', 'D_111', 'B_10', 'D_113', 'R_4', 'D_48', 'R_26', 'S_23', 'B_11', 'D_104', 'D_134', 'D_79', 'P_3', 'D_132', 'D_137', 'D_135', 'B_28', 'D_88', 'S_13', 'D_51', 'D_61', 'D_75', 'D_69', 'R_16', 'S_6', 'S_17', 'D_93', 'B_20', 'D_112', 'D_123', 'D_130', 'B_1', 'D_78', 'D_92', 'S_27', 'D_44', 'B_16', 'R_5', 'D_43', 'S_18', 'B_15', 'D_39', 'D_50', 'D_55', 'S_9', 'D_105', 'D_70', 'R_18', 'D_125', 'D_58', 'S_24', 'D_110', 'D_42', 'R_6', 'D_81', 'R_7', 'D_138', 'D_52', 'R_27', 'D_124', 'D_45', 'D_91', 'D_108', 'S_22', 'B_14', 'D_83', 'R_13', 'D_87', 'S_19', 'D_131', 'R_21', 'B_40', 'R_3', 'D_65', 'B_13', 'D_129', 'D_119', 'B_32', 'R_9', 'B_24', 'D_127', 'D_106', 'D_102', 'R_1', 'R_14', 'B_37', 'D_107', 'R_15', 'R_12', 'P_4', 'R_10', 'D_121', 'R_11', 'S_11', 'D_76', 'D_143', 'B_39', 'B_42', 'D_62', 'B_4', 'S_26', 'D_77']
    

In [6]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

## TODO: FAST AMEX implementation is not accurate; convert the dataframe accurate version to directly work with numpy arrays
# @yunchonggan's fast metric implementation
# From https://www.kaggle.com/competitions/amex-default-prediction/discussion/328020
def fast_amex_metric(y_true: np.array, y_pred: np.array) -> float:

    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return 0.5 * (g + d)

# Need Lightgbm supported eval metric
#Custom eval function expects a callable with following signatures: func(y_true, y_pred), func(y_true, y_pred, weight) or func(y_true, y_pred, weight, group) and returns (eval_name, eval_result, is_higher_better) or list of (eval_name, eval_result, is_higher_better):
def lgbm_eval_metric_amex(y_true, y_pred):
    amex_metric = fast_amex_metric(y_true, y_pred)
    return ('amex', amex_metric, True)

In [7]:
fold_score_track = {}
def track_score(key, score):
    key = str(key)
    if key not in fold_score_track:
        fold_score_track[key] = []
    fold_score_track[key].append(score)

def show_score(key):
    key = str(key)
    display(HTML(f"<h3>{key} OVERALL SCORE : {np.mean(fold_score_track[key]):0.6f}</h3>"))

In [87]:
for dataset in ['train_data','train_data_emp1', 'test_data','test_data_emp1']:
    df = load_prepare_amex_dataset(dataset)
    nanmap = df.isna().sum().reset_index().rename({'index':'feature',0:'nan_count'}, axis=1).to_dict('records')
    print(dataset, "nan", len([x for x in nanmap if x['nan_count']>0]))
    del df
gc.collect()

train_data nan 122
train_data_emp1 nan 51
test_data nan 121
test_data_emp1 nan 50


0

In [9]:
dataset_nan_columns = {}
for dataset in ['train_data_emp1','test_data_emp1']:# ['train_data','train_data_emp1', 'test_data','test_data_emp1']:
    df = load_prepare_amex_dataset(dataset)
    nanmap = df.isna().sum().reset_index().rename({'index':'feature',0:'nan_count'}, axis=1).to_dict('records')
    print(dataset, "nan", len([x for x in nanmap if x['nan_count']>0]))
    dataset_nan_columns[dataset] = [x['feature'] for x in nanmap if x['nan_count']>0]
    del df
gc.collect()

train_data_emp1 nan 34
test_data_emp1 nan 37


0

In [20]:
test_only_missing = list(set(dataset_nan_columns['test_data_emp1']).difference(set(dataset_nan_columns['train_data_emp1'])))
test_only_missing

['D_86', 'S_12', 'S_17']

In [8]:
try_train_df = load_prepare_amex_dataset('test_data_emp1_nn')

In [9]:
nanmap = try_train_df.isna().sum().reset_index().rename({'index':'feature',0:'nan_count'}, axis=1).to_dict('records')
print("nan", len([x for x in nanmap if x['nan_count']>0]))

nan 3


In [11]:
del try_train_df
gc.collect()

0

In [60]:
variant_scored_df = load_prepare_amex_dataset(f"variant_scores_emp1")
variant_scored_df
variant_scored_df = variant_scored_df[~variant_scored_df['feature_variant'].str.contains('D_66')]
variant_scored_df.to_parquet(os.path.join(BS_PATH,'variant_scores_emp1.parquet'),index=False)

In [10]:
[x for x in nanmap if x['nan_count']>0]

[{'feature': 'S_12', 'nan_count': 2385},
 {'feature': 'S_17', 'nan_count': 12535},
 {'feature': 'D_86', 'nan_count': 1067}]

In [37]:
variant_scores_df = load_prepare_amex_dataset(f"variant_scores_emp1")
variant_scores_df = variant_scores_df.sort_values(['score','feature_variant'], ascending=False)
# variant_scored_df
# variant_scored_df = variant_scored_df[~variant_scored_df['feature_variant'].str.contains('D_64')]

In [61]:
variant_scores_df[variant_scores_df['feature_variant'].str.contains('D_66')]

,feature_variant,score


In [62]:
variant_scores_df

,feature_variant,score
331,P_2_global_max,0.638529
332,P_2_global_median,0.638485
329,P_2_global_mean,0.638406
330,P_2_global_min,0.638382
333,P_2_local_mean,0.638281
...,...,...
861,D_110_local_mean,0.021387
862,D_110_local_linear_interpolate,0.021387
860,D_110_global_median,0.021387
857,D_110_global_mean,0.021387


In [41]:
feature_variants = load_prepare_amex_dataset(f"train_data_D_116")
feature_variants

,customer_ID,D_116,D_116_global_most_frequent,D_116_global_least_frequent,D_116_global_unknown,D_116_local_most_frequent,D_116_local_least_frequent,D_116_local_nearest_interpolate_mf,D_116_local_nearest_interpolate_lf
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
5531446,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5531447,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5531448,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5531449,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# the df we refer to here are the emp1 version, so df_train is df_train_emp1
# df_train_nan_mask is NaNs in original df


df_train_nan_mask = load_prepare_amex_dataset("train_data_nan_mask")
df_test_nan_mask = load_prepare_amex_dataset("test_data_nan_mask")
df_train_rows = len(df_train_nan_mask)
df_test_rows = len(df_test_nan_mask)
df_all_nan_mask = pd.concat([df_train_nan_mask, df_test_nan_mask],ignore_index=True)
gc.collect()

526

In [31]:
nan_features_df = df_train_nan_mask.sum().reset_index().rename({'index':'feature',0:'nan_count'}, axis=1)
len(nan_features_df[nan_features_df['nan_count']>0]['feature'].tolist())

121

In [32]:
nan_features_df = df_test_nan_mask.sum().reset_index().rename({'index':'feature',0:'nan_count'}, axis=1)
len(nan_features_df[nan_features_df['nan_count']>0]['feature'].tolist())

120

In [14]:
nan_features_df = df_all_nan_mask.sum().reset_index().rename({'index':'feature',0:'nan_count'}, axis=1)
missingvaluefeatures = nan_features_df[nan_features_df['nan_count']>0]['feature'].tolist()
len(missingvaluefeatures)
missingvaluefeature = missingvaluefeatures[0]

In [15]:
df_train_train, df_train_test, df_test_train, df_test_test = None, None, None, None
for dataset in ['train','test']:
    thedf = load_prepare_amex_dataset(f"{dataset}_data_emp1")
    thedf['index'] = thedf.index
    thedf['dataset'] = f"{dataset}"
    exec(f"df_{dataset}_train = thedf.loc[~df_{dataset}_nan_mask[missingvaluefeature]]")
    exec(f"df_{dataset}_test = thedf.loc[df_{dataset}_nan_mask[missingvaluefeature]]")
    del thedf
    gc.collect()

In [16]:
df_all_train = pd.concat([df_train_train, df_test_train],ignore_index=True)
del df_train_train, df_test_train
gc.collect()

0

In [22]:
def create_lgbm_model_with_config(random_state=1, n_estimators=5000, importance_type=None, early_stopping_rounds=None, modeltype="classifier"):
    """
    Creates model with our desired and some default hyper params
    importance_type: split|gain
    """
    if early_stopping_rounds is None:
        early_stopping_rounds = n_estimators//10
    if modeltype == "classifier":
        return LGBMClassifier(n_estimators=n_estimators,
                              learning_rate=0.09, reg_lambda=50,
                              min_child_samples=2400,
                              num_leaves=95, num_threads=12,
                              colsample_bytree=0.19,early_stopping_rounds=early_stopping_rounds,
                              max_bins=511, random_state=random_state, importance_type=importance_type)
    if modeltype == "regressor":
        return LGBMRegressor(n_estimators=n_estimators,
                              learning_rate=0.03, reg_lambda=50,
                              min_child_samples=2400,
                              num_leaves=95, num_threads=12,
                              colsample_bytree=0.19,early_stopping_rounds=early_stopping_rounds,
                              max_bins=511, random_state=random_state, importance_type=importance_type)


In [11]:
gc.collect()

0

In [43]:
variant_scored_df = load_prepare_amex_dataset(f"variant_scores_emp1")
variant_scored_df

,feature_variant,score
0,R_7,0.191363
1,R_7_global_mean,0.191363
2,R_7_global_min,0.191363
3,R_7_global_max,0.191363
4,R_7_global_median,0.191363
...,...,...
963,D_68_global_unknown,0.159087
964,D_68_local_most_frequent,0.155616
965,D_68_local_least_frequent,0.155923
966,D_68_local_nearest_interpolate_mf,0.156455


In [46]:
variant_scored_df = load_prepare_amex_dataset(f"variant_scores_emp1")
variant_scored_df
# variant_scored_df = variant_scored_df[~variant_scored_df['feature_variant'].str.contains('D_64')]
# variant_scored_df.to_parquet(os.path.join(BS_PATH,'variant_scores_emp1.parquet'),index=False)

In [23]:
%%time
# variant_df = variant_df[reversed(list(variant_df.columns))]
# we keep this copy so that original feature doens't stay on top!!!
# it's some weird quirk i found
# variant_df[missingvaluefeature+"_COPY"] = variant_df[missingvaluefeature]
# del variant_df[missingvaluefeature]

features_x = [f for f in df_all_train.columns if f not in ['customer_ID', 'target', 'S_2', 'index','dataset',missingvaluefeature]]
feature_y = missingvaluefeature
# df_x = df_train[features_x]
# df_y = df_train_wt[feature_y]
# target =
target = df_all_train[missingvaluefeature]

total_splits = 5

# kf = StratifiedKFold(n_splits=total_splits, shuffle=True)
kf = KFold(n_splits = total_splits, shuffle=True)
fold_scores = []
models = []
# NOT TOO SURE about feature_importances when all features are variants!! it's non decisive
for fold, (idx_train, idx_dev) in enumerate(kf.split(df_all_train, target)):
    print(f"------ FOLD: {fold+1} of {total_splits} -------")
    train_x, dev_x, train_y, dev_y, model = None, None, None, None, None
    start_time = datetime.now()

    X_tr = df_all_train.iloc[idx_train][features_x]
    X_va = df_all_train.iloc[idx_dev][features_x]
    y_tr = target[idx_train]
    y_va = target[idx_dev]
   
    model = create_lgbm_model_with_config(modeltype='regressor',importance_type='gain')
    model.fit(X_tr, y_tr,
                  eval_set = [(X_va, y_va)], 
                  eval_metric=['l2'],
                  callbacks=[log_evaluation(10)]
             )
    n_trees = model.best_iteration_
    if n_trees is None: n_trees = model.n_estimators
  
    models.append(model)
  
    

------ FOLD: 1 of 5 -------


MemoryError: Unable to allocate 18.6 GiB for an array with shape (13431480, 186) and data type float64

In [85]:
gc.collect()

18763

In [23]:
%%time
# df_test = load_prepare_amex_dataset("test_data_emp1")
# df_test_last1 = df_test.groupby('customer_ID').last()
# df_test_last1 = df_test_last1.reset_index()
# df_test_wt = pd.merge(df_test_last1, train_label_df, how='inner', on = 'customer_ID')#.reset_index()

CPU times: total: 9min 3s
Wall time: 8min 2s


In [59]:
features_x = [f for f in df_test.columns if f not in ['customer_ID', 'target', 'prediction', 'S_2']]
X_tr = df_test[features_x]

predictions = []
for model in models:
    preds = model.predict_proba(X_tr)
    predictions.append(preds[:,1])#/len(models)
    


924621

In [66]:
df_test_last1['prediction'] = np.max(predictions, axis=0)
df_test_last1[['customer_ID','prediction']].to_csv("submission.csv",index=False)

```
Data: EMP0
Features: similar aggregation like https://www.kaggle.com/code/ambrosm/amex-lightgbm-quickstart, except Cat aggs
Model: 5 fold, lgbm standard
Ensemble: choose max prediction value;highest proba out of 5 models
```